In [1]:
import logging
import os

# set the directory to the location of the script
try:
    os.chdir("..")
    target_directory = os.getenv(
        "TARGET_DIRECTORY", os.getcwd()
    )  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

### Access Operational DB (Post Analyics) -> CosmosDB

In [6]:
from src.cosmosdb.manager import CosmosDBMongoCoreManager

cosmos_manager = CosmosDBMongoCoreManager(
    connection_string=os.getenv("AZURE_COSMOS_CONNECTION_STRING"),
    database_name=os.getenv("AZURE_COSMOS_DB_DATABASE_NAME"),
    collection_name=os.getenv("AZURE_COSMOS_DB_COLLECTION_NAME"),
)

In [7]:
query = {"session_id": "b40e326e"}
document = cosmos_manager.read_document(query)

if document:
    print("Found session:", document)
else:
    print("No session found.")

Found session: {'_id': 'b40e326e', 'session_id': 'b40e326e', 'timestamp': '2025-05-26T22:18:15Z', 'history': [{'role': 'system', 'content': '\n\n"""ROLE & CONTEXT"""\nYou are RTMedAgent, a HIPAA‑compliant, real‑time healthcare voice assistant\nserving patients of XYZ Health Clinic after multi‑factor authentication.\n\n"""CALL METADATA"""\n• patient_id: P54321\n• patient_name: Alice Brown\n• patient_dob: 1987-04-12\n• patient_phone_number: 5552971078\n\n"""DATA VALIDATION"""\nIf any CALL METADATA field above (patient_name, patient_dob, or patient_phone_number) is missing, politely ask the patient to provide or confirm the missing information before proceeding.\n\n"""INTENT ONTOLOGY (MUST CHOOSE ONE)"""\n1. schedule_appointment\n2. change_appointment\n3. cancel_appointment\n4. refill_prescription\n5. fill_new_prescription\n6. get_current_prescriptions\n7. lookup_medication_info\n8. lookup_side_effects\n9. check_drug_interactions\n10. evaluate_prior_authorization\n11. transfer_medication_

## Access Cache (redis) 

In [12]:
from usecases.RTMedAgent.backend.orchestration.conversation_state import ConversationManager
from src.redis.manager import AzureRedisManager
redis_manager = AzureRedisManager()

2025-05-26 17:24:43,671 - micro - MainProcess - INFO     Azure Redis connection initialized. (manager.py:__init__:41)
INFO:micro:Azure Redis connection initialized.


In [13]:
cm = ConversationManager.from_redis("b40e326e", redis_manager)

2025-05-26 17:24:49,415 - micro - MainProcess - INFO     Restored session b40e326e: 15 msgs, ctx keys=['authenticated', 'latency_roundtrip'] (conversation_state.py:from_redis:41)
INFO:micro:Restored session b40e326e: 15 msgs, ctx keys=['authenticated', 'latency_roundtrip']


Templates found: ['voice_agent_authentication.jinja', 'voice_agent_system.jinja']


In [16]:
cm.hist[:100]

[{'role': 'system',
  'content': '\n\n"""ROLE & CONTEXT"""\nYou are RTMedAgent, a HIPAA‑compliant, real‑time healthcare voice assistant\nserving patients of XYZ Health Clinic after multi‑factor authentication.\n\n"""CALL METADATA"""\n• patient_id: P54321\n• patient_name: Alice Brown\n• patient_dob: 1987-04-12\n• patient_phone_number: 5552971078\n\n"""DATA VALIDATION"""\nIf any CALL METADATA field above (patient_name, patient_dob, or patient_phone_number) is missing, politely ask the patient to provide or confirm the missing information before proceeding.\n\n"""INTENT ONTOLOGY (MUST CHOOSE ONE)"""\n1. schedule_appointment\n2. change_appointment\n3. cancel_appointment\n4. refill_prescription\n5. fill_new_prescription\n6. get_current_prescriptions\n7. lookup_medication_info\n8. lookup_side_effects\n9. check_drug_interactions\n10. evaluate_prior_authorization\n11. transfer_medication_pharmacy\n12. insurance_billing_question\n13. escalate_emergency\n14. general_health_question\n15. unknown 

In [14]:
lat = cm.get_context("latency_roundtrip", {})

for stage, entries in lat.items():
    # extract all the durations
    durations = [e["dur"] for e in entries]

    # compute summary stats
    count   = len(durations)
    average = sum(durations) / count if count else 0
    minimum = min(durations) if count else 0
    maximum = max(durations) if count else 0

    # pretty-print
    print(f"⏱️ Stage “{stage}” — "
          f"calls: {count}, "
          f"avg: {average:.3f}s, "
          f"min: {minimum:.3f}s, "
          f"max: {maximum:.3f}s")

⏱️ Stage “tts” — calls: 12, avg: 0.001s, min: 0.001s, max: 0.003s
⏱️ Stage “processing” — calls: 5, avg: 1.476s, min: 0.737s, max: 2.044s
